In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# File paths
census_tract_shp = 'la_census_tract/LA_City_2020_Census_Tracts_.shp'
crime_csv = 'crime_data/crime_la_2020_2024.csv'

# Generate list of years from 2001 to 2024
years = list(range(2010, 2024))

In [3]:
# Read in the census tract shapefile
tracts_gdf = gpd.read_file(census_tract_shp)
tracts_gdf = tracts_gdf.to_crs(epsg=4326)
tracts_gdf = tracts_gdf[['geometry', 'CT20']]

# Read in the crime points data, this is very large dataset
crime_df = pd.read_csv(crime_csv)

In [4]:
# Convert the date to a datetime object
crime_df['Date Rptd'] = pd.to_datetime(crime_df['Date Rptd'])
crime_df['Year'] = crime_df['Date Rptd'].dt.year

/var/folders/6g/xqgvnv4s1h95kfz0bd14h3_h0000gn/T/ipykernel_68276/3146834728.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crime_df['Date Rptd'] = pd.to_datetime(crime_df['Date Rptd'])


In [5]:
# Define funtion to aggreate crime points counts by census tract, by type of crime
def aggregate_points(points_gdf, geometry_gdf):
    # Spatial join the points to the census tracts
    points_gdf = gpd.sjoin(points_gdf, geometry_gdf, how='inner', predicate='within')
    # Group by the census tract and
    points_gdf = points_gdf.groupby(['CT20', 'Crm Cd']).size().reset_index(name='count')
    return points_gdf
    

In [6]:
# Create an empty dataframe to store the combined results
combined_results = pd.DataFrame()

# Loop through each year
for year in years:
    # Filter the crime data for the current year
    crime_df_year = crime_df[crime_df['Year'] == year]
    
    # Convert the filtered crime data to a GeoDataFrame
    crime_gdf_year = gpd.GeoDataFrame(crime_df_year, geometry=gpd.points_from_xy(crime_df_year.LON, crime_df_year.LAT))
    crime_gdf_year = crime_gdf_year.set_crs(epsg=4326)
    
    # Aggregate the crime points by census tract
    aggregated_points = aggregate_points(crime_gdf_year, tracts_gdf)
    
    # Add the year to the aggregated points dataframe
    aggregated_points['Year'] = year
    
    # Append the results to the combined dataframe
    combined_results = pd.concat([combined_results, aggregated_points], ignore_index=True)

In [7]:
# Create a 'geoid10' columns. goeid10 = '06037' + CT20
combined_results['geoid10'] = '06037' + combined_results['CT20'].astype(str)
combined_results = combined_results[['geoid10', 'Crm Cd', 'count', 'Year']]

# Replace the comma in 'Crm Cd Desc' with ' -'
# combined_results['Crm Cd Desc'] = combined_results['Crm Cd Desc'].str.replace(',', '-')

# Save the combined results to a CSV file
combined_results.to_csv('crime_data/crime_la_2020_2024_by_tract_type.csv', index=False)

# Randomly sample 10,000 rows from the combined results, and display the first 5 rows.
combined_results.sample(10000).head()

,geoid10,Crm Cd,count,Year
49958,06037191302,121,1,2021
35350,06037101122,420,3,2021
65166,06037262200,745,5,2021
142207,06037274100,210,6,2023
9934,06037135204,745,1,2020
